## Imports

In [ ]:
from pycaret.classification import load_model, predict_model 
import pandas as pd 
import numpy as np
import streamlit as st
from PIL import Image
import os

## Streamlit

In [ ]:
class StreamlitApp:
    
    def __init__(self):
        self.model = load_model('box_office_model') 
        self.save_fn = 'path.csv' 
        
    def predict(self, input_data): 
    return predict_model(self.model, data=input_data)
    
    def store_prediction(self, output_df): 
        if os.path.exists(self.save_fn):
            save_df = pd.read_csv(self.save_fn)
            save_df = save_df.append(output_df, ignore_index=True)
            save_df.to_csv(self.save_fn, index=False)
            
        else: 
            output_df.to_csv(self.save_fn, index=False) 
            
    